# 1. Example to grap images from a webcam
(but this only gives you an rgb, not depth)
hit 'q' in the window to stop

In [1]:
import cv2 as cv
cap = cv.VideoCapture(0)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Our operations on the frame come here
    img = frame
    #img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    if len(frame)>0:cv.imshow('img', img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [2]:
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

# 2. Example to convert simulated RGB-D images into point clouds

In [ ]:
import os
os._exit(0)

In [1]:
import sys
sys.path.append('../../../robotics-course/build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/quim/stg/jungsu/robotics-course/course3-Simulation/04-opencv'** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '


**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}
** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/quim/stg/jungsu/robotics-course/course3-Simulation/04-opencv'


4.2.0** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}




In [10]:
#-- Add REAL WORLD configuration and camera
RealWorld = ry.Config()
RealWorld.addFile("../../../robotics-course/scenarios/challenge.g")
RealWorld.frame("obj1").setColor([1,0,0])
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")


C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
D = C.view()
cameraFrame = C.frame("camera")

In [11]:
# Intrinsic Params..
camInfo = cameraFrame.info()
f = camInfo['focalLength']
f = f * camInfo['height']
fxfypxpy = [f, f, camInfo['width']/2, camInfo['height']/2]

In [12]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
            
        if len(rgb)>0: cv.imshow('OPENCV - rgb',
                                 cv.cvtColor(rgb, cv.COLOR_BGR2RGB))
        if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)

In [13]:
cv.destroyAllWindows()

# 3. example to use multiple camera attached to different robot frames

In [ ]:
import os
os._exit(0)

In [ ]:
import sys
sys.path.append('../../build')
import libry as ry
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")
# change the position of the central sensor
f = RealWorld.frame("camera")
f.setPosition(f.getPosition()+[0,0,.5])
# add a frame for the additional camera
f = RealWorld.addFrame("R_gripperCamera", "R_gripperCenter")
f.setShape(ry.ST.marker, [.2])
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera") # camera is a pre-existing frame that specifies the intrinsic camera parameter
S.addSensor("Rcamera", "R_gripperCamera", 640, 360, 1.) # R_gripperCamera is a fresh frame - we have to specify intrinsic parameters explicitly
for _ in range(100):
    S.step([], 0.01, ry.ControlMode.none)

for k in range(5):
    # get images from the wrist
    S.selectSensor("camera")
    [rgb, depth] = S.getImageAndDepth()
    input()
    # get images from the main sensor
    S.selectSensor("Rcamera")
    [rgb, depth] = S.getImageAndDepth()
    input()